# Goal
* make fit generator than can use 'blob/idx' info

In [1]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, AveragePooling2D, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop, Adadelta, Adam, SGD
from keras.utils.np_utils import to_categorical
%cd ~/repos/ece1512_project/
import mrc, fit_generator_helper, customizable_deep_network

Using TensorFlow backend.


/Users/gw/repos/ece1512_project


In [44]:
from imp import reload
reload(fit_generator_helper)

<module 'fit_generator_helper' from '/Users/gw/repos/ece1512_project/fit_generator_helper.py'>

In [7]:
d = np.load('/Users/gw/repos/superpick/data/muhn_Data2_P30_J106/particles_selected.cs')
df = pd.DataFrame(d[['blob/path','alignments2D/class', 'blob/idx']])
pdir = '/Users/gw/repos/superpick/data/'
df['blob/path'] = pdir + df['blob/path'].str.decode("utf-8") #https://stackoverflow.com/questions/40389764/how-to-translate-bytes-objects-into-literal-strings-in-pandas-dataframe-pytho
df = df.rename(columns={'alignments2D/class':'class','blob/idx':'idx','blob/path':'fname'})
header = mrc.read_header(df['fname'].loc[0])
nx,ny = header['nx'], header['ny']
d = {k:v for v,k in enumerate(df['class'].unique())}
df['class'] = df['class'].map(d)
df = df.head(100)

In [8]:
# 10%/90% test/train split
val_n = int(0.1*df.shape[0]) 
df = df.iloc[:-val_n]
df_val = df.iloc[-val_n:]

In [9]:
X_val,Y_val = fit_generator_helper.XY_from_df_batch(df_val,nx=nx,ny=ny,num_classes=df.head(10)['class'].unique().size)

In [10]:
params = customizable_deep_network.architecture_params_wrapper(blocks=6,
                                inputl_kernel=30,
                                inputl_node=8,
                                hiddenl_kernels=(15,7,5),
                                hiddenl_nodes=((8,8),(8,16),(32,32)),
                                pooling_sizes = (15,7,5,3),
                                n1_ave_pool=64, 
                                n2_ave_pool=64, 
                                k1_ave_pool=3, 
                                k2_ave_pool=3, 
                                av_pool=4,
                                val_dense=256,
                                drop=0.5
                               )

In [11]:
# Define input shape and number of classes

input_shape=X_val.shape[1::]
n_classes = Y_val.shape[1]

# Call function build_network

model = customizable_deep_network.build_network (
  blocks=params['blocks'],
  input_shape=input_shape, 
  inputl_node=params['inputl_node'],
  inputl_kernel=params['inputl_kernel'],
  hiddenl_node=params['hiddenl_node'],
    
  hiddenl_kernel=params['hiddenl_kernel'], 
  pooling_size=params['pooling_size'], 
  n1_ave_pool=params['n1_ave_pool'], 
  n2_ave_pool=params['n2_ave_pool'], 
    
  k1_ave_pool=params['k1_ave_pool'], 
  k2_ave_pool=params['k2_ave_pool'], 
  av_pool=params['av_pool'], 
  val_dense=params['val_dense'], 
  drop=params['drop'], 
  n_classes=n_classes)

model.compile(optimizer = SGD() , loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 352, 352, 8)       7208      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 352, 352, 8)       57608     
_________________________________________________________________
batch_normalization_6 (Batch (None, 352, 352, 8)       32        
_________________________________________________________________
activation_6 (Activation)    (None, 352, 352, 8)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 176, 176, 8)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 176, 176, 8)       14408     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 176, 176, 8)       14408     
__________

# train model

In [12]:
batch_size = 10
steps_per_epoch = np.floor(df.shape[0] / batch_size) # 110 for 11k particles

In [13]:
for _ in range(1):
    print('epoch %i' % _)
    model.fit_generator(fit_generator_helper.image_loader(df,batch_size=batch_size,nx=nx,ny=ny,num_classes=7),
                    steps_per_epoch=steps_per_epoch, # steps_per_epoch is number of batches per epoch
                    epochs=1,
                   )
    scores = model.evaluate(X_val, Y_val)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

epoch 0
Epoch 1/1
10/10 [==============================] - 8s 754ms/step
categorical_accuracy: 40.00%
